In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from random import randint
from time import sleep
import csv
import numpy as np

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [12]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver')

In [14]:

#for the name of bps
bpsNamekab = []

with open("URL_NTB.csv","r",encoding='utf-8') as oj:
    oj_reader = csv.reader(oj, delimiter = ',')
    for row in oj_reader:
        bpsNamekab.append(row[7])
bpsNamekab.pop(0)

'URL'

In [13]:
finaldf = pd.DataFrame()

    #import the dataset detail
bpsNamedetail = pd.read_csv('URL_NTB.csv', encoding = 'utf-8')
#set the index
bpsNamedetail = bpsNamedetail.set_index('URL', drop = True)

In [16]:
import re

##we need looping here
nameLength = len(bpsNamekab)
f = open('hasil_NTB_mei.csv', 'w', encoding="utf-8")
with f:
    writer = csv.writer(f)
    for l in range(0, nameLength):


        driver.get(bpsNamekab[l])
        sleep(randint(8,12))



        #Find element by link text and store in variable "Element"
        try:
    
            Element = driver.find_element_by_xpath('//button[contains(@aria-label, "ulasan") and (@class = "widget-pane-link")]')
            
            
            #This will scroll the page till the element is found and click
            driver.execute_script("arguments[0].scrollIntoView();", Element)
            driver.execute_script('arguments[0].click();', Element)
                
            sleep(randint(5,8))

            #for the effective of scrollable bar
            a = bpsNamedetail['Reviews'].loc[bpsNamekab[l]]
            if(a <= 7 ):
                i = 1
            elif(a>= 8 and a<= 20):
                i = 3
            elif(a>20 and a<= 200):
                i = 27
            elif(a >200 and a<=400):
                i = 52
            elif(a>400 and a<= 600):
                i = 77
            elif(a>600 and a<=800):
                i = 102
            elif(a>800):
                i = 150

            for i in range(1,i):
                scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
                
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
                sleep(randint(2,3))
        except NoSuchElementException:
            "tidak ada"
    
        
        ##get number of comments
        nums = len(driver.find_elements_by_class_name("section-review-content"))
        print(nums)

        #to store the comments
        totalReview =  []
        totalRating = []
        mydf = pd.DataFrame(columns = ['review', 'rating'])

        commentss = driver.find_elements_by_class_name("section-review-content")
        
        for commentt in commentss:

            #for the reviews detail
            com_text  = commentt.find_element_by_class_name('section-review-text').text



            #check if the tags is exists using try/catch
            try:
                review_text1 = commentt.find_element_by_class_name("section-review-stars").get_attribute("aria-label")
                review_text = float(review_text1.split(' ')[1])
            except NoSuchElementException:
                "tidak ada"
            
            ##using another tags to grab the stars
            try:
                review_text1 = commentt.find_element_by_class_name("section-review-numerical-rating").text
                review_text2 = re.search(r"[0-9]+/", review_text1).group()
                review_text = re.sub("/", "", review_text2)
            except NoSuchElementException:
                "tidak ada"
      
            try:
                publish_date = commentt.find_element_by_class_name('section-review-publish-date-and-source').text
            except NoSuchElementException:
                "tidak ada"
            try:
                publish_date = commentt.find_element_by_class_name('section-review-publish-date').text
            except NoSuchElementException:
                "tidak ada"
            
            detail = list(bpsNamedetail.loc[bpsNamekab[l]])

            detail.append(str(com_text))
            detail.append(str(review_text))
            detail.append(str(publish_date))

               
            writer.writerow(detail)
                

KeyboardInterrupt: 